# Uso de dicionários para substituir códigos pelas categorias

## Links para dados de seguro de automóveis da SUSEP

* Autoseg: https://www2.susep.gov.br/menuestatistica/Autoseg/principal.aspx

* Dados agregados do segundo semestre de 2020: https://www2.susep.gov.br/redarq.asp?arq=Autoseg2021A%2ezip (é preciso descompactar o arquivo zip)


Sobre os dados completos:
* [Manual da SUSEP](https://www.gov.br/susep/pt-br/servicos/mercado/enviar-dados/arquivos/manual_de_orientacao_para_envio_de_dados_out2023.pdf) (p.81-. Sinistros: p.87-)


* Link para baixar os dados completos: ['AUTO_2021A.rar'](https://www2.susep.gov.br/download/basesdados/scd_auto/AUTO_2021A.rar)

Após baixar os dados completos no computador, eu descompactei o arquivo, o que gerou:
* R_AUTO_2021A: apólices em 2020
* S_AUTO_2021A: sinistros em 2020

Eu carreguei o arquivo 'S_AUTO_2021A.csv' para o meu *google drive*.

* Se você tiver dificuldades para baixar o arquivo .rar acima, tente baixar diretamente aqui: [S_AUTO_2021A.csv](https://drive.google.com/file/d/18a9l66HZpLjKRki45X9_udM6iFfGlozy/view?usp=drive_link)



Importar pacotes necessários:

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive, files
import os

pd.options.display.float_format = '{:.2f}'.format    # pandas: para todos os números aparecerem com duas casas decimais

## Leitura dos microdados

Os dados serão lidos a partir do *google drive*.

In [4]:
# alterar o caminho se necessário, especificando a pasta
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dados/AUTO_2021A')
# ler o arquivo csv
# o conjunto de dados está com o separador ; e os decimais com ,
df = pd.read_csv('S_AUTO_2021A.csv', sep=';', decimal=',')

Mounted at /content/drive


<ipython-input-4-a06ec60eebe7>:6: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('S_AUTO_2021A.csv', sep=';', decimal=',')


## Visão geral dos dados


In [5]:
# para visualizar até o número desejado de colunas
pd.set_option('display.max_columns', 24)

In [6]:
# mostrar as 5 primeiras linhas
df.head()

,cod_apo,ENDOSSO,ITEM,MODALIDADE,TIPO_PROD,COBERTURA,COD_MODELO,ANO_MODELO,COD_TARIF,REGIAO,COD_CONT,EVENTO,INDENIZ,VAL_SALVAD,D_SALVADO,VAL_RESS,D_RESS,D_AVI,D_LIQ,D_OCORR,CAUSA,SEXO,D_NASC,CEP
0,NaN,0000000000,0,1,1,1,004354-0,2009,10,11,1,8,63.00,0.00,0,0.00,0,20201016,20201026,20201015,7,M,19740216,9371420
1,NaN,0000000000,0,1,1,1,004354-0,2009,10,11,1,8,63.00,0.00,0,0.00,0,20201017,20201026,20201016,7,M,19740216,9371420
2,NaN,0000000000,0,1,1,1,004354-0,2009,10,11,1,8,80.00,0.00,0,0.00,0,20201017,20201105,20201016,7,M,19740216,9030620
3,NaN,0000000000,0,1,1,1,004354-0,2009,10,11,1,8,100.00,0.00,0,0.00,0,20200812,20200819,20200811,7,M,19740216,9371520
4,NaN,0000000000,0,1,1,1,004354-0,2009,10,11,1,8,127.00,0.00,0,0.00,0,20200812,20200819,20200811,7,M,19740216,0


In [7]:
# número de linhas (observações) e colunas (variáveis)
df.shape

(3747647, 24)

Há informações sobre 3.747.647 observações (sinistros) e são 24 variáveis.

## Ajustes já vistos

In [8]:
# cópia do dataframe
dados = df.copy()
# apagar variáveis
dados = dados.drop(['cod_apo', 'ENDOSSO'], axis=1)
# apagar linha faltante
dados.dropna(inplace=True)

In [9]:
# limpeza da variável 'REGIAO'
# converter todas as ocorrências para número e considerar as não númericas como NaN
dados['REGIAO'] = pd.to_numeric(dados['REGIAO'], errors='coerce')
# transformar NaN em 0
dados['REGIAO'].fillna(0, inplace=True)
# retirar linhas com 'REGIAO' igual a 0 e com 'CEP' igual a 0
dados = dados.drop(dados[(dados['REGIAO'] == 0) & (dados['CEP'] == 0)].index)

<ipython-input-9-90fbab4f458b>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['REGIAO'].fillna(0, inplace=True)


In [10]:
# nova coluna 'DATA' pela transformação da data de nascimento no formato correto
dados['DATA'] = pd.to_datetime(dados['D_NASC'], format='%Y%m%d', errors='coerce')
# criar nova coluna 'IDADE' calculada a partir de 'DATA'
now = pd.to_datetime('now', utc=True)
dados['IDADE'] = (now.year - dados['DATA'].dt.year) - ((now.month - dados['DATA'].dt.month) < 0)

In [11]:
# quantos NaN temos na coluna IDADE?
dados['IDADE'].isna().sum()

np.int64(556912)

In [12]:
# vamos substituí-las por 'Não informada'
dados['IDADE'] = dados['IDADE'].fillna('Não informada')

In [13]:
dados.shape

(3747009, 24)

In [14]:
# dados.sample(10)

# Uso de dicionários para substituir códigos por categorias

* Para que possamos substituir os códigos de algumas variáveis pelas suas ocorrências, vamos criar dicionários.

* O dicionário é um tipo de dado em que temos o par 'chave: valor', sendo que a chave tem que ser única para aquele dicionário.

* Para realizar as substituições que vou mostrar, é preciso baixar os dados referentes ao segundo semestre de 2020 e lá estarão algumas planilhas importantes.

In [15]:
dados.head()

,ITEM,MODALIDADE,TIPO_PROD,COBERTURA,COD_MODELO,ANO_MODELO,COD_TARIF,REGIAO,COD_CONT,EVENTO,INDENIZ,VAL_SALVAD,D_SALVADO,VAL_RESS,D_RESS,D_AVI,D_LIQ,D_OCORR,CAUSA,SEXO,D_NASC,CEP,DATA,IDADE
0,0,1,1,1,004354-0,2009,10,11.00,1,8,63.00,0.00,0,0.00,0,20201016,20201026,20201015,7,M,19740216,9371420,1974-02-16,51.00
1,0,1,1,1,004354-0,2009,10,11.00,1,8,63.00,0.00,0,0.00,0,20201017,20201026,20201016,7,M,19740216,9371420,1974-02-16,51.00
2,0,1,1,1,004354-0,2009,10,11.00,1,8,80.00,0.00,0,0.00,0,20201017,20201105,20201016,7,M,19740216,9030620,1974-02-16,51.00
3,0,1,1,1,004354-0,2009,10,11.00,1,8,100.00,0.00,0,0.00,0,20200812,20200819,20200811,7,M,19740216,9371520,1974-02-16,51.00
4,0,1,1,1,004354-0,2009,10,11.00,1,8,127.00,0.00,0,0.00,0,20200812,20200819,20200811,7,M,19740216,0,1974-02-16,51.00


## 'MODALIDADE'

Para a variável 'MODALIDADE':
* Os códigos de tipos de produto estão no manual da SUSEP;
* Como são poucos códigos, vamos criar o dicionário de forma manual;
* Realizar a substituição.

Manual da SUSEP:

Modalidade:
* 1: VMR - Valor de Mercado Referenciado
* 2: VR - Valor Determinado
* 3: RCF - Produtos com uma única cobertura de RCF (Cobertura para Responsabilidade Civil: danos causados a terceiros)
* 4: Seguro Popular

In [16]:
# criação do dicionário
mod_d = {1: 'VMR',
         2: 'VR',
         3: 'RCF',
         4: 'SP'}

In [17]:
# checar quantas ocorrências de cada COBERTURA existem
dados['MODALIDADE'].value_counts()

,count
MODALIDADE,
1,3673090
3,66011
2,7900
4,8


Não parece haver nenhum valor estranho. Vamos efetuar as substituições.

In [18]:
# substituir os códigos pelas descrições usando replace()
dados['MODALIDADE']= dados['MODALIDADE'].replace(mod_d)

In [19]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['MODALIDADE'].value_counts()

,count
MODALIDADE,
VMR,3673090
RCF,66011
VR,7900
SP,8


In [20]:
# visualizar o dataframe
dados.sample(10)

,ITEM,MODALIDADE,TIPO_PROD,COBERTURA,COD_MODELO,ANO_MODELO,COD_TARIF,REGIAO,COD_CONT,EVENTO,INDENIZ,VAL_SALVAD,D_SALVADO,VAL_RESS,D_RESS,D_AVI,D_LIQ,D_OCORR,CAUSA,SEXO,D_NASC,CEP,DATA,IDADE
359088,1,VMR,1,1,515141-4,2014,42,17.00,1,8,2800.00,0.00,0,0.00,0,20200925,20201014,20200924,9,0,0,39663000,NaT,Não informada
1382559,1,VMR,1,1,999999-9,2017,10,11.00,1,1,2103.00,0.00,0,0.00,0,20200702,20200710,20200610,4,0,0,1000000,NaT,Não informada
1960708,0,VMR,1,1,005249-3,2012,10,1.00,1,8,90.00,0.00,0,0.00,0,20201201,20201221,20201129,7,F,19590131,91330330,1959-01-31,66.00
1208684,0,VMR,1,1,005308-2,2010,10,11.00,1,1,20409.00,0.00,0,0.00,0,20201009,20201009,20201009,2,0,0,5798350,NaT,Não informada
2971953,1,VMR,1,1,003457-6,2020,10,40.00,1,1,994.00,0.00,0,0.00,0,20200629,20200707,20200627,9,0,0,77000000,NaT,Não informada
130314,0,VMR,1,1,003409-6,2015,10,11.00,1,8,112.00,0.00,0,0.00,0,20200802,20200817,20200801,7,F,19730414,3607040,1973-04-14,52.00
2641574,0,VMR,1,1,003419-3,2016,10,11.00,1,8,63.00,0.00,0,0.00,0,20200721,20200810,20200720,7,F,19770124,4206001,1977-01-24,48.00
2303595,0,VMR,1,1,003414-2,2018,10,11.00,1,8,90.00,0.00,0,0.00,0,20201218,20210108,20201217,7,M,19870512,4557010,1987-05-12,38.00
3323295,0,VMR,1,1,001004-9,2000,10,16.00,1,8,97.00,0.00,0,0.00,0,20201112,20201203,20201110,7,M,19440823,31340160,1944-08-23,80.00
2973385,1,VMR,2,1,024195-4,2014,10,9.00,1,8,849.00,0.00,0,0.00,0,20201110,20201204,20201109,9,M,19740530,8790020,1974-05-30,51.00


## 'TIPO_PROD'

Para a variável 'TIPO_PROD':
* Os códigos de tipos de produto estão no manual da SUSEP;
* Como são poucos códigos, vamos criar o dicionário de forma manual;
* Realizar a substituição.

Manual da SUSEP:

Tipo de produto:
* 1: Padrão
* 2: Perfil


In [21]:
# criação do dicionário
prod_d = {1: 'Padrão',
          2: 'Perfil'}

In [22]:
# checar quantas ocorrências de cada COBERTURA existem
dados['TIPO_PROD'].value_counts()

,count
TIPO_PROD,
1,1983108
2,1763901


Não parece haver nenhum valor estranho. Vamos efetuar as substituições.

In [23]:
# substituir os códigos pelas descrições usando replace()
dados['TIPO_PROD']= dados['TIPO_PROD'].replace(prod_d)

In [24]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['TIPO_PROD'].value_counts()

,count
TIPO_PROD,
Padrão,1983108
Perfil,1763901


In [25]:
# visualizar o dataframe
# dados.sample(10)

## 'COBERTURA'

Os códigos de cobertura estão no manual da SUSEP. Como são poucos códigos, vamos criar o dicionário de forma manual.

Obs.: A cobertura compreensiva abrange casos de colisão, incêndio e roubo. É a mais ofertada pelos corretores e também a mais procurada por quem quer fazer um seguro para o seu veículo. É importante saber que o serviço cobre somente danos causados ao veículo do segurado e não prejuízos materiais ou pessoais a terceiros.

In [26]:
# checar quantas ocorrências de cada COBERTURA existem
dados['COBERTURA'].value_counts()

,count
COBERTURA,
1,3602039
9,71694
4,48037
2,24998
5,171
3,70


Não parece haver nenhum valor estranho. Vamos efetuar as substituições.

In [27]:
cob_d = {1: 'Compreensiva',
         2: 'Incêndio e roubo',
         3: 'Incêndio',
         4: 'Indenização integral, colisão e roubo',
         5: 'Colisão e incêndio',
         9: 'Outros'}

In [28]:
# substituir os códigos pelas descrições usando replace()
dados['COBERTURA']= dados['COBERTURA'].replace(cob_d)

In [29]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['COBERTURA'].value_counts()

,count
COBERTURA,
Compreensiva,3602039
Outros,71694
"Indenização integral, colisão e roubo",48037
Incêndio e roubo,24998
Colisão e incêndio,171
Incêndio,70


In [30]:
# visualizar o dataframe
# dados.sample(10)

## 'COD_MODELO'

Para a variável 'COD_MODELO':
* Há uma planilha pronta com os códigos e as descrições ('auto2_vei.csv'), vamos carregá-la;
* Criar o dicionário de substituição a partir da planilha;
* Realizar a substituição usando map() e não replace() para agilizar;
* Alterar o nome da variável para 'MODELO'.

In [31]:
# alterar o nome da variável para MODELO
dados.rename(columns={'COD_MODELO': 'MODELO'}, inplace=True)

In [32]:
# checar quantas ocorrências de cada COD_MODELO existem
dados['MODELO'].value_counts()

,count
MODELO,
003408-8,49103
015088-6,48840
999999-9,45479
002111-3,43031
004424-5,41622
...,...
005215-9,1
509102-0,1
033039-6,1


In [33]:
# carregar a planilha com as codificações dos modelos de automóveis
vei = pd.read_csv('/content/drive/My Drive/dados/Autoseg2021A/auto2_vei.csv', sep=';', encoding='latin1')

In [34]:
# criar o dicionário usando a coluna 'CODIGO' como chave
# e a coluna 'GRUPO' para fazer a substituição
# poderia ser a coluna 'DESCRICAO'
vei_d = vei.set_index('CODIGO')['GRUPO'].to_dict()

In [35]:
# vei_d

In [36]:
# substituir os códigos pelas descrições usando map()
# já criando a nova variável
dados['MODELO']= dados['MODELO'].map(vei_d)

In [37]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['MODELO'].value_counts().head(10)

,count
MODELO,
HYUNDAI HB20,139039
GM CHEVROLET ONIX,127416
HONDA FIT,96771
TOYOTA COROLLA,96343
FORD KA 1.0,85395
RENAULT SANDERO,73478
GM CHEVROLET PRISMA,73122
FORD ECO SPORT,69723
VW VOLKSWAGEN GOL 1.0,68907


* A função map() é muito mais rápida do que a função replace();
* Mas com a map(), ao usar o dicionário para efetuar as substituições, se aparecer algum valor no dataframe que não está no dicionário, ela transforma esse valor em NaN;
* Podemos substituir o NaN por qualquer valor que seja interessante para nós.

In [38]:
# número de NaNs
dados['MODELO'].isna().sum()

np.int64(51294)

In [39]:
# vamos substituir os NaN por 'Outros'
dados['MODELO'] = dados['MODELO'].fillna('OUTROS')

In [40]:
# número de NaNs
dados['MODELO'].isna().sum()

np.int64(0)

In [41]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['MODELO'].value_counts().head(20)

,count
MODELO,
HYUNDAI HB20,139039
GM CHEVROLET ONIX,127416
OUTROS,100251
HONDA FIT,96771
TOYOTA COROLLA,96343
FORD KA 1.0,85395
RENAULT SANDERO,73478
GM CHEVROLET PRISMA,73122
FORD ECO SPORT,69723


Vamos ver a aparência do dataframe após as alterações:

In [42]:
dados.sample(10)

,ITEM,MODALIDADE,TIPO_PROD,COBERTURA,MODELO,ANO_MODELO,COD_TARIF,REGIAO,COD_CONT,EVENTO,INDENIZ,VAL_SALVAD,D_SALVADO,VAL_RESS,D_RESS,D_AVI,D_LIQ,D_OCORR,CAUSA,SEXO,D_NASC,CEP,DATA,IDADE
194128,0,VMR,Padrão,Compreensiva,BMW - TODOS,2020,23,6.00,1,1,9039.00,0.00,0,0.00,0,20201123,20210108,20201122,9,M,19740402,85904000,1974-04-02,51.00
3315701,0,VMR,Padrão,Compreensiva,HONDA CIVIC,2014,10,11.00,1,8,85.00,0.00,0,0.00,0,20201027,20201112,20201026,7,M,19651128,4361050,1965-11-28,59.00
1331647,0,VMR,Padrão,Compreensiva,FORD KA 1.0,2015,10,23.00,1,8,101.00,0.00,0,0.00,0,20200718,20200806,20200714,7,M,19550506,50730280,1955-05-06,70.00
2238113,0,VMR,Padrão,Compreensiva,FORD FIESTA ACIMA DE 1.0,2017,10,39.00,1,8,237.00,0.00,0,0.00,0,20201117,20201201,20201116,7,F,19770912,74860170,1977-09-12,47.00
2085610,19,VMR,Padrão,Compreensiva,FORD CARGO,2012,42,2.00,1,8,470.00,0.00,0,0.00,0,20201002,20201020,20201001,7,0,0,95098020,NaT,Não informada
1811453,1,VMR,Padrão,Compreensiva,FIAT DUCATO,2015,20,13.00,1,1,250.00,0.00,0,0.00,0,20201014,20201211,20201014,9,0,0,18013000,NaT,Não informada
3446379,1,VMR,Perfil,Compreensiva,HONDA HR-V,2016,10,21.00,1,1,351.00,0.00,0,0.00,0,20200822,20201028,20200822,9,M,19920512,48790000,1992-05-12,33.00
3220204,1,VMR,Perfil,Compreensiva,GM CHEVROLET COBALT,2019,10,11.00,1,8,1363.00,0.00,0,0.00,0,20200826,20200911,20200821,9,F,19660514,8461310,1966-05-14,59.00
1318450,0,VMR,Padrão,Compreensiva,VW VOLKSWAGEN GOLF,2014,10,13.00,1,8,100.00,0.00,0,0.00,0,20200714,20200805,20200713,7,F,19930420,18606780,1993-04-20,32.00
164077,0,VMR,Perfil,Compreensiva,FIAT STRADA,2018,14A,19.00,1,1,241.00,0.00,0,0.00,0,20200706,0,20200702,2,M,19650721,27320000,1965-07-21,59.00


## 'REGIAO'

In [43]:
# eu especifiquei as subpastas onde os arquivos se encontram
# dentro do meu google drive: /dados/Autoseg2021A/
reg = pd.read_csv('/content/drive/My Drive/dados/Autoseg2021A/auto_reg.csv', sep=';', encoding='latin1')

In [44]:
# criar o dicionário usando a coluna 'CODIGO' como chave
# e a coluna 'DESCRICAO' para fazer a substituição
reg_d = reg.set_index('CODIGO')['DESCRICAO'].to_dict()

In [45]:
reg_d

{1: 'RS - Met. Porto Alegre e Caxias do Sul',
 2: 'RS - Demais regiões',
 3: 'SC - Met. Florianópolis e Sul',
 4: 'SC - Oeste',
 5: 'SC - Blumenau e demais regiões',
 6: 'PR - F.Iguaþu-Medianeira-Cascavel-Toledo',
 7: 'PR - Met. Curitiba',
 8: 'PR - Demais regiões',
 9: 'SP - Vale do Paraíba e Ribeira',
 10: 'SP - Litoral Norte e Baixada Santista',
 11: 'SP - Met. de São Paulo',
 12: 'SP - Grande Campinas',
 13: 'SP - Ribeirão Preto e Demais Mun. de Campinas',
 14: 'MG - Triângulo mineiro',
 15: 'MG - Sul',
 16: 'MG - Met.BH-Centro Oeste-Zona Mata-C. Vertentes',
 17: 'MG - Vale do Aço-Norte-Vale Jequitinhonha',
 18: 'RJ - Met. do Rio de Janeiro',
 19: 'RJ - Interior',
 20: 'ES - Espírito Santo',
 21: 'BA - Bahia',
 22: 'SE - Sergipe',
 23: 'PE - Pernambuco',
 24: 'PB - Paraíba',
 25: 'RN - Rio Grande do Norte',
 26: 'AL - Alagoas',
 27: 'CE - Ceará',
 28: 'PI - Piaui',
 29: 'MA - Maranhão',
 30: 'PA - Pará',
 31: 'AM - Amazonas',
 32: 'AP - Amapá',
 33: 'RO - Rondônia',
 34: 'RR - Rora

In [46]:
# verificar as ocorrências de REGIAO antes de alterar
dados['REGIAO'].value_counts()

,count
REGIAO,
11.00,995738
13.00,412541
16.00,193870
18.00,184816
21.00,148604
12.00,130399
7.00,129540
9.00,116569
2.00,114680


In [47]:
# substituir os códigos pelas descrições
dados['REGIAO'].replace(reg_d, inplace=True)

<ipython-input-47-965db1481571>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['REGIAO'].replace(reg_d, inplace=True)


In [48]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['REGIAO'].value_counts()

,count
REGIAO,
SP - Met. de São Paulo,995738
SP - Ribeirão Preto e Demais Mun. de Campinas,412541
MG - Met.BH-Centro Oeste-Zona Mata-C. Vertentes,193870
RJ - Met. do Rio de Janeiro,184816
BA - Bahia,148604
SP - Grande Campinas,130399
PR - Met. Curitiba,129540
SP - Vale do Paraíba e Ribeira,116569
RS - Demais regiões,114680


## 'COD_TARIF' (do veículo)

In [49]:
# checar quantas ocorrências de cada valor antes de alterar
dados['COD_TARIF'].value_counts()

,count
COD_TARIF,
10,2392395
22,335267
23,173402
14A,126391
11,123401
...,...
18,1
89,1
91,1


Esses códigos não parecem ser do tipo inteiros. Vamos checar:

In [50]:
dados['COD_TARIF'].unique()

array(['10', '23', '14A', '11', '42', '50', '40', '20', '22', '21', ' 50',
       ' 41', ' 51', '14C', ' 10', ' 11', ' 22', ' 23', '00', '80', '30',
       '14B', '31', '84', '15', '17', ' 40', '52', '82', ' 42', '51',
       '94', '16', '41', '72', '60', '73', ' 21', '010', '011', ' 52',
       ' 20', ' 17', '021', '020', '062', '042', '019', '023', '099',
       '90', '58', '18', ' 15', ' 31', '62', '70', '98', '92', ' 30',
       '81', '95', '-1', '85', '43', '040', ' 80', '030', '022', '015',
       '094', '91', ' 43', ' 60', '058', '041', '017', '99', '96', '93',
       '89', ' 16', '97', ' 18', '071', '031', ' 90', '83', '86', '052',
       ' 91', '61', '070', '19', ' 62', ' 19', '016', '061', '068', ' 72',
       '063', '63', '68', '71'], dtype=object)

Há alguns problemas: aparecem alguns espaços e alguns zeros antes de alguns números.

In [51]:
# tirar alguns dos espaços que aparecem antes de alguns códigos
dados['COD_TARIF'] = dados['COD_TARIF'].str.split().str.join(' ')
# retirar o 0 do início de alguns números
dados['COD_TARIF'] = dados['COD_TARIF'].str.replace(r'^0+(\d*)$', r'\1', regex=True)

In [52]:
dados['COD_TARIF'].unique()

array(['10', '23', '14A', '11', '42', '50', '40', '20', '22', '21', '41',
       '51', '14C', '', '80', '30', '14B', '31', '84', '15', '17', '52',
       '82', '94', '16', '72', '60', '73', '62', '19', '99', '90', '58',
       '18', '70', '98', '92', '81', '95', '-1', '85', '43', '91', '96',
       '93', '89', '97', '71', '83', '86', '61', '68', '63'], dtype=object)

Como tiramos os zeros que estavam à esquerda, '00' se tornou '', mas ainda todas as ocorrências são strings. Vamos trocar essas ocorrências pela string '0'.

In [53]:
dados['COD_TARIF'] = dados['COD_TARIF'].replace('', '0')

In [54]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['COD_TARIF'].value_counts()

,count
COD_TARIF,
10,2487463
22,352181
23,177210
11,126782
14A,126391
40,76246
30,67418
20,63925
42,61396


A variável 'COD_TARIF' envolverá mais etapas:
* Como não há a planilha pronta com os códigos e as descrições, vou copiar o texto do manual da SUSEP (Tabela 6-4);
* Criar o dicionário de substituição manualmente;
* Alterar o nome da variável para 'CATEGORIA';
* Realizar todas as substituições de acordo com o dicionário criado usando a função map();

Obs.: Perceba que os códigos estão entre aspas porque não estão sendo lidos como inteiros.

In [55]:
tarif_d = {'10': 'Passeio Nacional',
          '11': 'Passeio Importado',
          '14A': 'Pick-ups leves Nacionais – exceto Kombi e Saveiro',
          '14B':  'Pick-ups leves Nacionais – somente Kombi',
          '14C':  'Pick-ups leves Nacionais – somente Saveiro',
          '15': 'Pick-ups leves Importados',
          '16': 'Modelos esportivos Nacionais',
          '17': 'Modelos esportivos Importados',
          '18': 'Modelos Especiais (Passeio) Nacionais',
          '19': 'Modelos Especiais (Passeio) Importados',
          '20': 'Pick-ups pesadas Carga Nacionais',
          '21': 'Pick-ups pesadas Carga Importados',
          '22': 'Pick-ups pesadas Pessoas Nacionais',
          '23': 'Pick-ups pesadas Pessoas Importado',
          '30': 'Bicicletas Motorizadas, Motocicletas, Motonetas com Reboque ou SIDE-CAR, VESPACAR Nacionais',
          '31': 'Bicicletas Motorizadas, Motocicletas, Motonetas com Reboque ou SIDE-CAR, VESPACAR Importados',
          '40': 'Caminhões Leves Nacionais (até 6,9 ton)',
          '41': 'Caminhões Leves Importados (até 6,9 ton)',
          '42': 'Caminhões Pesados Nacionais (acima de 7 ton – inclusive)',
          '43': 'Caminhões Pesados Importados (acima de 7 ton – inclusive)',
          '50': 'Rebocadores Nacionais',
          '51': 'Rebocadores Importados',
          '52': 'Reboques e Semi-Reboques Nacionais',
          '53': 'Reboques e Semi-Reboques Importados',
          '58': 'Ônibus e Microônibus com Cobrança de Frete Nacionais (exceto urbano com linha regular)',
          '59': 'Ônibus e Microônibus com Cobrança de Frete Importados (exceto urbano com linha regular)',
          '60': 'Ônibus e Microônibus com Cobrança de Frete Nacionais ( urbano com linha regular)',
          '61': 'Ônibus e Microônibus com Cobrança de Frete Importados (urbano com linha regular)',
          '62': 'Ônibus e Microônibus sem Cobrança de Frete Nacionais',
          '63': 'Ônibus e Microônibus sem Cobrança de Frete Importados',
          '68': 'Tratores e Máquinas de Uso Urbano Nacionais',
          '69': 'Tratores e Máquinas de Uso Urbano Importados',
          '70': 'Tratores e Máquinas de Uso Rural Nacionais',
          '71': 'Tratores e Máquinas de Uso Rural Importados',
          '72': 'Guinchos Nacionais (carro socorro)',
          '73': 'Guinchos Importados (carro socorro)',
          '80': 'Táxi Nacionais',
          '81': 'Táxi Importados',
          '82': 'Lotação, Transporte Solidário Nacionais',
          '83': 'Lotação, Transporte Solidário Importados',
          '84': 'Veículo Escolar Nacional (transporte escolar)',
          '85': 'Veículo Escolar Importado (transporte escolar)',
          '86': 'Viagem de Entrega dentro do Território Nacional – Nacional',
          '87': 'Viagem de Entrega dentro do Território Nacional – Importado',
          '88': 'Viagem de Entrega Exclusivamente para Países da América do Sul – Nacional',
          '89': 'Viagem de Entrega Exclusivamente para Países da América do Sul – Importado',
          '90': 'Locadoras Nacionais',
          '91': 'Locadoras Importados',
          '92': 'Veículos Funerários Nacionais',
          '93': 'Veículos Funerários Importados',
          '94': 'Ambulância',
          '95': 'Auto-Escolas',
          '96': 'Bombeiros',
          '97': 'Policiamento'}

In [56]:
# alterar o nome da variável para CATEGORIA
dados.rename(columns={'COD_TARIF': 'CATEGORIA'}, inplace=True)

In [57]:
# usar a função map() para efetuar as substituições
dados['CATEGORIA'] = dados['CATEGORIA'].map(tarif_d)

In [58]:
# checar quantas ocorrências de cada valor existem após a alteração
dados['CATEGORIA'].value_counts()

,count
CATEGORIA,
Passeio Nacional,2487463
Pick-ups pesadas Pessoas Nacionais,352181
Pick-ups pesadas Pessoas Importado,177210
Passeio Importado,126782
Pick-ups leves Nacionais – exceto Kombi e Saveiro,126391
"Caminhões Leves Nacionais (até 6,9 ton)",76246
"Bicicletas Motorizadas, Motocicletas, Motonetas com Reboque ou SIDE-CAR, VESPACAR Nacionais",67418
Pick-ups pesadas Carga Nacionais,63925
Caminhões Pesados Nacionais (acima de 7 ton – inclusive),61396


In [59]:
# como usamos a função map(), é possível que NaNs tenham sido gerados
# quantos NaN temos na coluna CATEGORIA?
dados['CATEGORIA'].isna().sum()

np.int64(46630)

In [60]:
# vamos substituí-los por 'Outros'
dados['CATEGORIA'] = dados['CATEGORIA'].fillna('Outros')

In [61]:
# quantos NaN sobraram?
dados['CATEGORIA'].isna().sum()

np.int64(0)

In [62]:
# dados.sample(10)

Tarefa 2: Usar dicionários para substituir as ocorrências das variáveis 'COD_CONT' e 'EVENTO':
- Use o manual da SUSEP de forma similar às apresentadas para encontrar os significados dos códigos para cada uma delas.
- Crie os dicionários de forma manual.
- Efetue as substituições.
- Para cada uma das variáveis use a função value_counts() para verificar as contagens
  